In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import os 

In [2]:
ct_file = "../Task1/image/left_knee.nii.gz"
mask_file = "..//Task1/output/bone_segmentation_task1_1.nii.gz"
output_folder = "segmented_regions"

In [ ]:
def visualize(image_data,name):
    # --- Choose slices to visualize ---
    num_slices = 9
    slice_indices = np.linspace(0, image_data.shape[2] - 1, num_slices, dtype=int)

    # --- Plot slices ---
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    axes = axes.flatten()

    for i, slice_idx in enumerate(slice_indices):
        axes[i].imshow(image_data[:, :, slice_idx], cmap='gray')
        axes[i].set_title(f"Slice {slice_idx}")
        axes[i].axis('off')
    plt.savefig(name)
    plt.tight_layout()
    plt.show()

def load_nifti(file_path):
    """
    Load a NIfTI file and return the data array and affine.
    """
    nifti = nib.load(file_path)
    data = nifti.get_fdata()
    affine = nifti.affine
    return data, affine


def save_nifti(data, affine, output_path):
    """
    Save a NumPy array as a NIfTI file.
    """
    nifti = nib.Nifti1Image(data.astype(np.float32), affine)
    nib.save(nifti, output_path)

In [ ]:
image_data,_=load_nifti(ct_file)
visualize(image_data,'test')

In [ ]:
def segment_knee_regions(ct_path, mask_path, output_dir):
    """
    Segment Tibia, Femur, and Background from a CT volume using the mask.
    """
    # Load the CT scan and segmentation mask
    ct_data, ct_affine = load_nifti(ct_path)
    mask_data, _ = load_nifti(mask_path)

    # Define region labels (change if your mask uses different values)
    TIBIA_LABEL = 1
    FEMUR_LABEL = 2
    BACKGROUND_LABEL = 0

    # Generate binary masks
    tibia_mask = (mask_data == TIBIA_LABEL)
    femur_mask = (mask_data == FEMUR_LABEL)
    background_mask = (mask_data == BACKGROUND_LABEL)

    # Apply masks to CT data
    tibia_volume = ct_data * tibia_mask
    femur_volume = ct_data * femur_mask
    background_volume = ct_data * background_mask

    # Save each region as a new .nii.gz file
    os.makedirs(output_dir, exist_ok=True)
    save_nifti(tibia_volume, ct_affine, os.path.join(output_dir, "tibia_volume.nii.gz"))
    save_nifti(femur_volume, ct_affine, os.path.join(output_dir, "femur_volume.nii.gz"))
    save_nifti(background_volume, ct_affine, os.path.join(output_dir, "background_volume.nii.gz"))

    print("Segmentation done. Files saved to:", output_dir)


In [ ]:
segment_knee_regions(ct_file, mask_file, output_folder)

In [ ]:
def overlay_mask(image_slice, mask_slice, alpha=0.4, cmap_img='gray', cmap_mask='jet'):
    """
    Overlay a segmentation mask on a single image slice.
    
    Parameters:
    - image_slice: 2D numpy array of the image slice.
    - mask_slice: 2D numpy array of the mask slice (same size as image_slice).
    - alpha: float, transparency of the mask overlay.
    - cmap_img: str, colormap for the image.
    - cmap_mask: str, colormap for the mask.
    """
    plt.imshow(image_slice, cmap=cmap_img)
    if mask_slice is not None and np.any(mask_slice):
        plt.imshow(mask_slice, cmap=cmap_mask, alpha=alpha)
    plt.axis('off')

def visualize_nifti_with_mask(image_path, mask_path=None, num_slices=9, axis=2):
    """
    Visualize NIfTI image slices with optional mask overlays.
    
    Parameters:
    - image_path: str, path to the image .nii.gz file.
    - mask_path: str or None, path to the mask .nii.gz file (optional).
    - num_slices: int, number of slices to visualize.
    - axis: int, axis along which to slice (0=sagittal, 1=coronal, 2=axial).
    """
    # Load image and optional mask
    image, ct_affine = load_nifti(image_path)
    mask, _ = load_nifti(mask_path)
    assert mask is None or image.shape == mask.shape, "Image and mask must have the same shape"

    # Select slices evenly along the chosen axis
    slice_indices = np.linspace(0, image.shape[axis] - 1, num_slices, dtype=int)
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    axes = axes.flatten()

    for i, idx in enumerate(slice_indices):
        ax = axes[i]
        plt.sca(ax)

        if axis == 0:
            img_slice = image[idx, :, :]
            msk_slice = mask[idx, :, :] if mask is not None else None
        elif axis == 1:
            img_slice = image[:, idx, :]
            msk_slice = mask[:, idx, :] if mask is not None else None
        else:  # default to axial
            img_slice = image[:, :, idx]
            msk_slice = mask[:, :, idx] if mask is not None else None

        overlay_mask(img_slice, msk_slice)
        ax.set_title(f"Slice {idx}")
    plt.savefig('slices with masks ')
    plt.tight_layout()
    plt.show()


In [ ]:
visualize_nifti_with_mask(
    ct_file,
    mask_file,  # Set to None if you don’t have a mask
    num_slices=9,
    axis=2  # axial view
)

In [ ]:
def plot_image_mask_overlay(image_path, mask_path, slice_index, axis=2, alpha=0.4):
    """
    Plot image slice, mask slice, and overlay side by side.

    Parameters:
    - image: 3D numpy array (CT volume)
    - mask: 3D numpy array (segmentation mask)
    - slice_index: int, index of the slice to visualize
    - axis: int, slicing axis (0=sagittal, 1=coronal, 2=axial)
    - alpha: float, transparency for overlay
    """
    image, ct_affine = load_nifti(image_path)
    mask, _ = load_nifti(mask_path)
    # Extract slices
    if axis == 0:
        img_slice = image[slice_index, :, :]
        mask_slice = mask[slice_index, :, :]
    elif axis == 1:
        img_slice = image[:, slice_index, :]
        mask_slice = mask[:, slice_index, :]
    else:
        img_slice = image[:, :, slice_index]
        mask_slice = mask[:, :, slice_index]

    # Plotting
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    axs[0].imshow(img_slice, cmap='gray')
    axs[0].set_title(f'Image Slice {slice_index}')
    axs[0].axis('off')

    axs[1].imshow(mask_slice, cmap='jet')
    axs[1].set_title(f'Mask Slice {slice_index}')
    axs[1].axis('off')

    axs[2].imshow(img_slice, cmap='gray')
    axs[2].imshow(mask_slice, cmap='jet', alpha=alpha)
    axs[2].set_title('Overlay')
    axs[2].axis('off')
    plt.savefig('slice_108_with_maska')
    plt.tight_layout()
    plt.show()

In [ ]:
slice_index = 108  
axis = 2  # Axial view

plot_image_mask_overlay(ct_file, mask_file, slice_index, axis=axis)

In [1]:
pip install torch torchvision

  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 14.5 MB/s eta 0:00:0000:0100:01
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [torchvision] [torchvision]
Note: you may need to restart the kernel to use updated packages.


In [24]:
import torch
import torch.nn as nn
import torchvision.models as models
import math

# --- Helper Functions for Inflation ---

def inflate_conv(conv2d, kernel_depth):
    """Inflates a Conv2d layer to a Conv3d layer by repeating weights."""
    if conv2d.in_channels % conv2d.groups != 0 or conv2d.out_channels % conv2d.groups != 0:
         raise ValueError("Inflating grouped convolutions is not straightforward with simple repetition.")

    # Get 2D parameters
    in_channels = conv2d.in_channels
    out_channels = conv2d.out_channels
    kernel_size_2d = conv2d.kernel_size
    stride_2d = conv2d.stride
    padding_2d = conv2d.padding
    dilation_2d = conv2d.dilation
    groups = conv2d.groups
    bias = conv2d.bias is not None

    # Create 3D convolution parameters
    # Temporal kernel size
    kernel_size_3d = (kernel_depth, kernel_size_2d[0], kernel_size_2d[1])
    
    # Typically stride 1 in time for simple inflation unless downsampling is desired
    # Based on the previous I3D code structure, transitions downsample spatially,
    # and the initial conv might downsample spatially. Temporal downsampling is handled
    # by the pooling layers in transition blocks.
    # So we'll use a temporal stride of 1 here for convolution inflation.
    stride_3d = (1, stride_2d[0], stride_2d[1])
    # Temporal padding to keep temporal dimension size
    padding_3d = (kernel_depth // 2, padding_2d[0], padding_2d[1])
    dilation_3d = (1, dilation_2d[0], dilation_2d[1]) # Dilation usually 1 in time

    # Create 3D Conv layer
    conv3d = nn.Conv3d(in_channels, out_channels, kernel_size_3d, stride_3d, padding_3d,
                       dilation=dilation_3d, groups=groups, bias=bias)

    # Inflate weights (replication and scaling)
    conv2d_weights = conv2d.weight.data #  Shape [out_channels, 3, H, W]
    #averaged_weights = conv2d_weights.mean(dim=1, keepdim=True) # Shape [out_channels, 1, H, W]

                 # Now repeat this 1-channel weight along the temporal dimension
    inflated_weights = conv2d_weights.unsqueeze(2).repeat(1, 1, kernel_depth, 1, 1)



    # Normalize by dividing by the depth (as per requirement ii)
    if kernel_depth > 0: # Avoid division by zero if somehow kernel_depth is 0
        inflated_weights = inflated_weights / kernel_depth

    # Copy inflated weights to the 3D Conv layer
    conv3d.weight.data.copy_(inflated_weights)

    # Copy bias if it exists
    if bias:
        conv3d.bias.data.copy_(conv2d.bias.data)

    return conv3d

def inflate_batch_norm(bn2d):
    """Inflates a BatchNorm2d layer to a BatchNorm3d layer."""
    bn3d = nn.BatchNorm3d(bn2d.num_features)
    # Copy parameters and running statistics
    bn3d.weight.data.copy_(bn2d.weight.data)
    bn3d.bias.data.copy_(bn2d.bias.data)
    bn3d.running_mean.copy_(bn2d.running_mean)
    bn3d.running_var.copy_(bn2d.running_var)
    # bn3d.num_batches_tracked.copy_(bn2d.num_batches_tracked) # Copy if your BN uses this
    return bn3d

def inflate_relu(relu2d):
    """Returns a ReLU layer (same for 2D and 3D)."""
    return nn.ReLU(inplace=relu2d.inplace)

def inflate_pool(pool2d, temporal_stride=1):
    """Inflates a Pooling layer to a 3D Pooling layer."""
    # Get 2D parameters
    kernel_size_2d = pool2d.kernel_size
    stride_2d = pool2d.stride
    padding_2d = pool2d.padding
    dilation_2d = pool2d.dilation if hasattr(pool2d, 'dilation') else 1 # MaxPool2d has dilation
    return_indices = pool2d.return_indices if hasattr(pool2d, 'return_indices') else False # MaxPool2d has this
    ceil_mode = pool2d.ceil_mode if hasattr(pool2d, 'ceil_mode') else False # Pool2d has this

    # Ensure kernel_size, stride, padding are tuples for consistency
    if not isinstance(kernel_size_2d, tuple): kernel_size_2d = (kernel_size_2d,) * 2
    if not isinstance(stride_2d, tuple): stride_2d = (stride_2d,) * 2
    if not isinstance(padding_2d, tuple): padding_2d = (padding_2d,) * 2
    if not isinstance(dilation_2d, tuple): dilation_2d = (dilation_2d,) * 2


    # Create 3D pooling parameters
    # Temporal kernel size (1 for no pooling in time by the pool layer itself, temporal_stride handles downsampling)
    kernel_size_3d = (1, kernel_size_2d[0], kernel_size_2d[1])
    # Temporal stride for downsampling
    stride_3d = (temporal_stride, stride_2d[0], stride_2d[1])
    # Temporal padding (0 as we don't pool over time)
    padding_3d = (0, padding_2d[0], padding_2d[1])
    dilation_3d = (1, dilation_2d[0], dilation_2d[1])

    if isinstance(pool2d, nn.MaxPool2d):
        return nn.MaxPool3d(kernel_size_3d, stride=stride_3d, padding=padding_3d,
                            dilation=dilation_3d, return_indices=return_indices, ceil_mode=ceil_mode)
    elif isinstance(pool2d, nn.AvgPool2d):
        # AvgPool2d also has count_include_pad attribute
        count_include_pad = pool2d.count_include_pad if hasattr(pool2d, 'count_include_pad') else True
        return nn.AvgPool3d(kernel_size_3d, stride=stride_3d, padding=padding_3d, ceil_mode=ceil_mode,
                            count_include_pad=count_include_pad) # Count include pad might need adjustment for 3D?
    else:
        raise ValueError(f"Unsupported pooling type: {type(pool2d)}")

# --- Inflated DenseNet Components ---

class InflatedDenseLayer(nn.Module):
    def __init__(self, dense_layer2d, conv_kernel_depth=3):
        super(InflatedDenseLayer, self).__init__()
        self.layers = nn.Sequential()
        for name, child in dense_layer2d.named_children():
            if isinstance(child, nn.BatchNorm2d):
                self.layers.add_module(name, inflate_batch_norm(child))
            elif isinstance(child, nn.ReLU):
                self.layers.add_module(name, inflate_relu(child))
            elif isinstance(child, nn.Conv2d):
                # For the bottleneck 1x1 conv, kernel_depth is 1
                # For the 3x3 conv, kernel_depth is user-specified (default 3)
                self.layers.add_module(name, inflate_conv(child, kernel_depth=child.kernel_size[0] if child.kernel_size != (1,1) else 1))

            else:
                raise ValueError(f"Unsupported layer type in DenseLayer: {type(child)}")
        # DenseLayer also has a drop_rate attribute
        self.drop_rate = dense_layer2d.drop_rate

    def forward(self, x):
        new_features = self.layers(x)
        if self.drop_rate > 0 and self.training: # Apply dropout only during training
            new_features = nn.functional.dropout(new_features, p=self.drop_rate, training=self.training)
        # DenseNet connectivity: concatenate input with new features
        return torch.cat([x, new_features], 1)

class InflatedTransition(nn.Module):
    def __init__(self, transition2d, temporal_pool_stride=2):
        super(InflatedTransition, self).__init__()
        self.layers = nn.Sequential()
        for name, child in transition2d.named_children():
            if isinstance(child, nn.BatchNorm2d):
                self.layers.add_module(name, inflate_batch_norm(child))
            elif isinstance(child, nn.ReLU):
                self.layers.add_module(name, inflate_relu(child))
            elif isinstance(child, nn.Conv2d): # This is the 1x1 convolution
                 self.layers.add_module(name, inflate_conv(child, kernel_depth=1))
            elif isinstance(child, nn.AvgPool2d): # This is the pooling layer for downsampling
                self.layers.add_module(name, inflate_pool(child, temporal_stride=temporal_pool_stride))
            else:
                raise ValueError(f"Unsupported layer type in Transition: {type(child)}")

    def forward(self, x):
        return self.layers(x)

# --- Main Inflation Function ---

def inflate_densenet121(densenet2d, frame_nb, conv_kernel_depth=3, temporal_pool_stride=2,input_channels=1):
    """
    Inflates a torchvision DenseNet121 model to a 3D model.

    Args:
        densenet2d (torchvision.models.densenet.DenseNet): The pre-trained 2D DenseNet121 model.
        frame_nb (int): The expected number of frames in the input video.
        conv_kernel_depth (int): The temporal kernel size to use for inflating 2D convs > 1x1.
                                 Default is 3 (3x3x3).
        temporal_pool_stride (int): The temporal stride to use for inflating spatial pooling
                                    layers in transition blocks. Default is 2.

    Returns:
        torch.nn.Module: The inflated 3D DenseNet model.
    """
    # Inflate the features part (contains initial conv, pool, dense blocks, transitions)
    features_3d = nn.Sequential()
    transition_nb = 0
    # Flag to identify the very first convolutional layer
    is_first_conv_layer = True
    for name, child in densenet2d.features.named_children():
        if isinstance(child, nn.Conv2d): # Initial Conv2d
            if is_first_conv_layer:
                 # --- SPECIAL HANDLING FOR THE VERY FIRST CONV LAYER ---
                 # The first Conv2d in DenseNet121 has in_channels=3.
                 # We need to change its in_channels to 'input_channels' (e.g., 1)
                 # and adapt its weights.

                 original_conv2d = child
                 original_in_channels = original_conv2d.in_channels # This is 3
                 if original_in_channels != 3:
                     # This inflation logic assumes the original model starts with 3 channels
                     warnings.warn(f"Expected first Conv2d to have 3 input channels, but got {original_in_channels}. Inflation logic might be incorrect.")

                 # Get original parameters except in_channels
                 out_channels = original_conv2d.out_channels
                 kernel_size_2d = original_conv2d.kernel_size
                 stride_2d = original_conv2d.stride
                 padding_2d = original_conv2d.padding
                 dilation_2d = original_conv2d.dilation
                 groups = original_conv2d.groups
                 bias = original_conv2d.bias is not None

                 # Create the new Conv3d with the DESIRED input_channels
                 # The temporal kernel size for the first layer is typically the spatial kernel size (7x7 -> 7x7x7)
                 kernel_depth_3d = kernel_size_2d[0] # Use spatial kernel size for time

                 kernel_size_3d = (kernel_depth_3d, kernel_size_2d[0], kernel_size_2d[1])
                 stride_3d = (1, stride_2d[0], stride_2d[1]) # Temporal stride 1
                 padding_3d = (kernel_depth_3d // 2, padding_2d[0], padding_2d[1])
                 dilation_3d = (1, dilation_2d[0], dilation_2d[1])

                 first_conv3d = nn.Conv3d(input_channels, out_channels, kernel_size_3d, stride_3d, padding_3d,
                                          dilation=dilation_3d, groups=groups, bias=bias)

                 # Inflate weights: Original weights shape [out_channels, 3, H, W]
                 original_weights = original_conv2d.weight.data

                 if input_channels == 1 and original_in_channels == 3:
                     # Average the original 3 input channel weights
                     adapted_weights_2d = original_weights.mean(dim=1, keepdim=True) # Shape [out_channels, 1, H, W]
                 elif input_channels == original_in_channels:
                     # Input channels match, just use original weights
                     adapted_weights_2d = original_weights
                 else:
                     # Handle other input channel numbers if needed (e.g., randomly initialize new weights)
                     warnings.warn(f"Handling inflation from {original_in_channels} to {input_channels} channels. "
                                   "Using averaged weights if original=3, otherwise weights might need custom handling.")
                     if original_in_channels == 3:
                         adapted_weights_2d = original_weights.mean(dim=1, keepdim=True).repeat(1, input_channels, 1, 1) # Repeat averaged weight
                     else:
                         # Fallback: Random initialization for the new layer if sizes don't match
                         print(f"Initializing weights for first Conv3d ({original_in_channels} -> {input_channels}) randomly.")
                         # The first_conv3d layer is already initialized randomly by default, so nothing more to do here.
                         # We can just skip the weight copying step.
                         adapted_weights_2d = None # Indicate no specific weights to copy


                 if adapted_weights_2d is not None:
                    # Repeat the adapted 2D weights along the temporal dimension
                    # Shape [out_channels, input_channels, 1, H, W] -> [out_channels, input_channels, D, H, W]
                    inflated_weights = adapted_weights_2d.unsqueeze(2).repeat(1, 1, kernel_depth_3d, 1, 1)

                    # Normalize by dividing by the depth
                    if kernel_depth_3d > 0:
                         inflated_weights = inflated_weights / kernel_depth_3d

                    # Copy inflated weights to the new Conv3d
                    first_conv3d.weight.data.copy_(inflated_weights)

                 if bias:
                     first_conv3d.bias.data.copy_(original_conv2d.bias.data)

                 features_3d.add_module(name, first_conv3d)
                 is_first_conv_layer = False # Mark that the first conv is processed

            else:
                 # --- STANDARD INFLATION FOR SUBSEQUENT CONV LAYERS ---
                 # These layers should use the standard inflate_conv logic
                 # Their in_channels will match the out_channels of the preceding 3D layer.
                 # We still need to handle the kernel depth for non-1x1 convs.
                 temporal_k_depth = child.kernel_size[0] if child.kernel_size != (1,1) else 1
                 features_3d.add_module(name, inflate_conv_standard(child, kernel_depth=temporal_k_depth))

             # The initial convolution in DenseNet121 is 7x7. Inflate it.
            
        elif isinstance(child, nn.BatchNorm2d): # Initial BatchNorm
             features_3d.add_module(name, inflate_batch_norm(child))
        elif isinstance(child, nn.ReLU): # Initial ReLU
             features_3d.add_module(name, inflate_relu(child))
        elif isinstance(child, nn.MaxPool2d): # Initial MaxPooling
             # Initial pool typically reduces spatial dimensions but not temporal
             # We'll make the temporal stride 1 here
             features_3d.add_module(name, inflate_pool(child, temporal_stride=1))
        elif isinstance(child, models.densenet._DenseBlock):
             # Inflate the DenseBlock
             block_3d = nn.Sequential()
             for nested_name, nested_child in child.named_children():
                 # Each child in a DenseBlock is a DenseLayer
                 assert isinstance(nested_child, models.densenet._DenseLayer)
                 block_3d.add_module(nested_name, InflatedDenseLayer(nested_child, conv_kernel_depth=conv_kernel_depth))
             features_3d.add_module(name, block_3d)
        elif isinstance(child, models.densenet._Transition):
             # Inflate the Transition layer
             features_3d.add_module(name, InflatedTransition(child, temporal_pool_stride=temporal_pool_stride))
             transition_nb += 1
        else:
            # print(f"Warning: Skipping unhandled layer type in features: {name} ({type(child)})")
            pass # Skip layers like OrderedDictWrapper if they appear

    # Calculate the final temporal dimension
    # Assumes each transition block reduces temporal dimension by temporal_pool_stride
    temporal_reduction_factor = int(math.pow(temporal_pool_stride, transition_nb))
    final_time_dim = frame_nb // temporal_reduction_factor
    if frame_nb % temporal_reduction_factor != 0:
         warnings.warn(f"Input frame_nb ({frame_nb}) is not perfectly divisible by temporal reduction factor ({temporal_reduction_factor}). "
                       "Final temporal dimension will be floor division result.")


    # Inflate the classifier part
    classifier_3d = nn.Sequential()
    for name, child in densenet2d.classifier.named_children():
        if isinstance(child, nn.Linear):
            # The linear layer input size needs to account for the flattened 3D features
            # It's final_time_dim * original_classifier_in_features
            original_in_features = child.in_features
            inflated_in_features = final_time_dim * original_in_features

            # Create new 3D linear layer
            linear3d = nn.Linear(inflated_in_features, child.out_features, bias=child.bias is not None)

            # Inflate weights
            linear2d_weights = child.weight.data # Shape [out_features, in_features_2d]

            # Reshape 2D weights to [out_features, in_features_2d, 1] (add a temporal dimension)
            inflated_weights = linear2d_weights.unsqueeze(2)
            # Repeat along the new temporal dimension
            inflated_weights = inflated_weights.repeat(1, 1, final_time_dim)
            # Reshape to match the 3D linear layer's expected shape [out_features, inflated_in_features]
            inflated_weights = inflated_weights.view(child.out_features, inflated_in_features)

            linear3d.weight.data.copy_(inflated_weights)

            # Copy bias if it exists
            if child.bias is not None:
                linear3d.bias.data.copy_(child.bias.data)

            classifier_3d.add_module(name, linear3d)

        else:
            # print(f"Warning: Skipping unhandled layer type in classifier: {name} ({type(child)})")
            pass # DenseNet's classifier is typically just a Linear layer

    # Combine features and classifier into a new Sequential model
    # We need to manually handle the final pooling and flatten step in the forward pass
    # like in the previous I3D code, as Sequential doesn't do this automatically.
    # So we'll create a custom module for the final model.

    class InflatedDenseNetModel(nn.Module):
        def __init__(self, features_3d, classifier_3d, final_time_dim, final_layer_nb):
            super().__init__()
            self.features = features_3d
            self.classifier = classifier_3d
            self.final_time_dim = final_time_dim
            self.final_layer_nb = final_layer_nb # This is the number of channels before global pooling

        def forward(self, x):
            x = self.features(x)
            # Apply ReLU after features (matches typical DenseNet flow before classification)
            x = nn.functional.relu(x)
            # Global spatial average pooling. Kernel size matches expected output spatial dims.
            # Assuming 7x7 spatial dims before classification for DenseNet121 on ImageNet size inputs
            spatial_kernel_h = x.shape[-2]
            spatial_kernel_w = x.shape[-1]
            x = nn.functional.avg_pool3d(x, kernel_size=(1, spatial_kernel_h, spatial_kernel_w))
            # Flatten for classifier
            # Original shape: [batch, channels, depth, 1, 1] after spatial pooling
            # Permute to [batch, depth, channels, 1, 1]
            x = x.permute(0, 2, 1, 3, 4).contiguous()
            # View to [batch, depth * channels]
            x = x.view(-1, self.final_time_dim * self.final_layer_nb)
            # Pass through classifier
            x = self.classifier(x)
            return x

    # Get the number of channels before the original 2D classifier
    final_layer_nb = densenet2d.classifier.in_features

    return InflatedDenseNetModel(features_3d, classifier_3d, final_time_dim, final_layer_nb)


# --- Testing the Inflation ---

import warnings

# a. Take a 2D pretrained DenseNet121 model
model_2d = models.densenet121(pretrained=True)
print("Loaded 2D DenseNet121 model.")

# Expected number of frames in your video input
input_frame_nb = 16
# Temporal kernel depth for inflating 3x3 convs
conv_k_depth = 3
# Temporal stride for pooling in transition blocks
pool_t_stride = 2

# Inflate the model
i3d_densenet_model = inflate_densenet121(
    model_2d,
    frame_nb=input_frame_nb,
    conv_kernel_depth=conv_k_depth,
    temporal_pool_stride=pool_t_stride
)

print(f"\nInflated 3D DenseNet model created for {input_frame_nb} frames.")
# Print the structure of the inflated model (optional, can be long)
# print(i3d_densenet_model)


# Create a dummy 3D input tensor
# Shape: [batch_size, channels, depth, height, width]
batch_size = 2
channels = 1
input_height = 224
input_width = 224

dummy_input_3d = torch.randn(batch_size, channels, input_frame_nb, input_height, input_width)
print("\nDummy input shape:", dummy_input_3d.shape)


# Perform a forward pass with the dummy input
try:
    i3d_densenet_model.eval() # Set model to evaluation mode (disables dropout, uses running stats for BN)
    with torch.no_grad(): # Disable gradient calculation for inference
        output = i3d_densenet_model(dummy_input_3d)

    print("\nForward pass successful!")
    print("Output shape:", output.shape)
    # The output shape should be [batch_size, num_classes]
    # For DenseNet121 pre-trained on ImageNet, num_classes is 1000
    print("Expected output shape:", (batch_size, 1000))

except Exception as e:
    print(f"\nForward pass failed: {e}")

Loaded 2D DenseNet121 model.

Inflated 3D DenseNet model created for 16 frames.

Dummy input shape: torch.Size([2, 1, 16, 224, 224])

Forward pass successful!
Output shape: torch.Size([2, 2048])
Expected output shape: (2, 1000)


NameError: name 'densenet2d' is not defined

## Task3

In [ ]:
import torch
import torch.nn as nn
import nibabel as nib
import numpy as np
import os
import warnings
import math
import pickle # For saving the dictionary
# Assuming your inflate_densenet121 function and helpers are defined here or imported
# from your previous work.
# from i3d_densenet_inflation import inflate_densenet121
import torchvision.models as models # Needed to load the 2D model


# --- (Include the inflate_conv, inflate_batch_norm, inflate_relu, inflate_pool,
#      InflatedDenseLayer, InflatedTransition, InflatedDenseNetModel, inflate_densenet121
#      definitions from previous responses here) ---
# Make sure you have the CORRECTED inflate_densenet121 that handles input_channels=1
# and the inflate_conv_standard helper.


# --- Helper function to find specific convolution layers (same as before) ---

def find_convolution_layers(module):
    """Recursively finds all Conv3d layers within a PyTorch module."""
    conv_layers = []
    for name, child in module.named_children():
        if isinstance(child, nn.Conv3d):
            conv_layers.append((name, child))
        else:
            conv_layers.extend(find_convolution_layers(child))
    return conv_layers

# --- Feature Extraction Function for a Specific Masked Region (same as before) ---
# This function remains the same as in the previous response, as its job is to
# process one region given the volume data and mask data.

def extract_features_from_masked_region(model_3d, volume_data, mask_data, region_name, frame_nb, target_layers):
    """
    Extracts a region from volume_data using mask_data, preprocesses it,
    runs it through the 3D CNN features, and extracts GAP features.

    Args:
        model_3d (torch.nn.Module): The inflated 3D DenseNet model.
        volume_data (np.ndarray): NumPy array of the loaded NIfTI volume data [D, H, W] or [D, H, W, C].
        mask_data (np.ndarray): NumPy array of the loaded NIfTI mask data for this region [D, H, W].
        region_name (str): The name of the region (e.g., 'Tibia').
        frame_nb (int): The expected number of frames/slices (CNN's depth) after preprocessing.
                         This should match the `frame_nb` used during inflation.
        target_layers (list): List of strings specifying which convolution layers
                              to extract features from.

    Returns:
        dict: A dictionary where keys are the target layer names/indices
              and values are the corresponding N-dimensional feature vectors
              after Global Average Pooling. Returns an empty dictionary if the
              region is not found in the mask or processing fails.
    Raises:
        ValueError: If input shapes are inconsistent or preprocessing fails.
        RuntimeError: If a forward pass fails.
    """
    print(f"      Processing region: {region_name}")

    # Ensure volume and mask shapes are compatible (ignoring potential channel dim in volume)
    volume_spatial_shape = volume_data.shape[:3] # Get D, H, W ignoring Channel
    mask_spatial_shape = mask_data.shape
    if volume_spatial_shape != mask_spatial_shape:
        raise ValueError(f"Volume and mask spatial shapes for region {region_name} do not match: {volume_spatial_shape} vs {mask_spatial_shape}")

    # Find the bounding box of the masked region
    coords = np.argwhere(mask_data > 0) # Assuming mask has non-zero values within the region
    if coords.shape[0] == 0:
        print(f"        Region {region_name} not found (mask is all zeros). Skipping.")
        return {} # Return empty features for this region

    min_coords = coords.min(axis=0)
    max_coords = coords.max(axis=0)

    # Add a buffer to the bounding box (optional, but often useful)
    buffer = 5 # Example buffer
    min_coords = np.maximum([0, 0, 0], min_coords - buffer)
    max_coords = np.minimum(volume_spatial_shape, max_coords + buffer + 1) # +1 to include max_coords

    # Crop the original volume data using the bounding box
    if volume_data.ndim == 4: # If original volume had shape [D, H, W, C]
         cropped_data = volume_data[min_coords[0]:max_coords[0],
                                    min_coords[1]:max_coords[1],
                                    min_coords[2]:max_coords[2], :]
         num_channels = cropped_data.shape[-1]
         print(f"        Cropped region has {num_channels} channels.")
    else: # Assuming grayscale [D, H, W]
         cropped_data = volume_data[min_coords[0]:max_coords[0],
                                    min_coords[1]:max_coords[1],
                                    min_coords[2]:max_coords[2]]
         num_channels = 1


    # --- Preprocessing the Cropped Region Data ---
    # This is a critical step: the cropped region must match the CNN's expected input dimensions
    current_region_depth = cropped_data.shape[0] # Assuming depth is the first dim
    target_depth = frame_nb

    if current_region_depth != target_depth:
         warnings.warn(f"        Cropped region depth ({current_region_depth}) does not match target frame_nb ({target_depth}). Applying padding/cropping.")
         if current_region_depth < target_depth:
             pad_amount = target_depth - current_region_depth
             pad_tuple = ((0, pad_amount), (0, 0), (0, 0))
             if cropped_data.ndim == 4:
                  pad_tuple = ((0, pad_amount), (0, 0), (0, 0), (0, 0))
             cropped_data = np.pad(cropped_data, pad_tuple, mode='constant', constant_values=0)
         elif current_region_depth > target_depth:
             start = (current_region_depth - target_depth) // 2
             cropped_data = cropped_data[start:start + target_depth, :, :]

    # Implement spatial resizing here if needed.
    # target_spatial_size = (224, 224) # Example target spatial size (H, W)
    # if cropped_data.shape[1:3] != target_spatial_size:
    #      warnings.warn(f"        Spatial dimensions of cropped region {cropped_data.shape[1:3]} do not match target spatial size {target_spatial_size}. Resizing needed.")
    #      # Example resizing (requires skimage)
    #      # from skimage.transform import resize
    #      # resize_shape = (cropped_data.shape[0], target_spatial_size[0], target_spatial_size[1])
    #      # if cropped_data.ndim == 4: resize_shape += (cropped_data.shape[3],) # Add channel if present
    #      # # Resize each channel/slice individually if needed, or use a 4D/5D resize function
    #      # # Be very careful with interpolation for medical data! Order 1 or 0 (nearest neighbor) often preferred.
    #      # resized_data = resize(cropped_data, resize_shape, order=1, preserve_range=True, anti_aliasing=False)
    #      # cropped_data = resized_data.astype(np.float32)
    #      pass


    # Convert preprocessed data to PyTorch tensor
    if cropped_data.ndim == 3: # Grayscale [D, H, W] after preprocessing
         input_tensor = torch.from_numpy(cropped_data).unsqueeze(0).unsqueeze(0) # -> [1, 1, D, H, W]
    elif cropped_data.ndim == 4: # [D, H, W, C] after preprocessing
         input_tensor = torch.from_numpy(cropped_data).permute(3, 0, 1, 2).unsqueeze(0) # -> [1, C, D, H, W]
    else:
         raise ValueError(f"Unexpected shape after preprocessing: {cropped_data.shape}")

    print(f"        Preprocessed tensor shape: {input_tensor.shape}")

    # --- Feature Extraction ---

    # Find all Conv3d layers and their names (assuming model_3d.features holds the CNN backbone)
    all_conv_layers = find_convolution_layers(model_3d.features)
    if not all_conv_layers:
        print(f"        No Conv3d layers found in the model's features. Skipping {region_name}.")
        return {}

    # Map target layer names/indices to actual layers
    layers_to_extract = {}
    num_conv_layers = len(all_conv_layers)
    target_layers = ['last', 'third_last', 'fifth_last']
    for layer_name, layer_module in all_conv_layers:
        if layer_name in target_layers:
            layers_to_extract[layer_name] = (layer_name, layer_module)
            found = True
            break

    if not found:
        warnings.warn(f"Target layer '{target}' not found by name.")

    if not layers_to_extract:
        print(f"        No valid target layers specified or found for region {region_name}. Skipping.")
        return {}

    # Hook to capture intermediate feature maps
    feature_maps = {}
    hooks = []

    def hook_fn(module, input, output):
         layer_info = None
         for name, layer_module in layers_to_extract.values():
              if layer_module is module:
                  layer_info = (name, type(layer_module).__name__)
                  break
         if layer_info:
              feature_maps[layer_info] = output.clone() # Capture output


    for key, (layer_name, layer_module) in layers_to_extract.items():
         hooks.append(layer_module.register_forward_hook(hook_fn))

    # Set model to evaluation mode and disable gradients
    model_3d.eval()
    with torch.no_grad():
        try:
            # Pass the preprocessed region tensor through the feature extraction layers
            _ = model_3d.features(input_tensor)
        except Exception as e:
            for h in hooks: h.remove()
            raise RuntimeError(f"      Forward pass failed for region {region_name}: {e}")

    for h in hooks:
        h.remove()

    # Apply Global Average Pooling (GAP) to extracted feature maps
    extracted_features = {}
    for layer_info, feature_map in feature_maps.items():
        if feature_map.dim() != 5:
             warnings.warn(f"        Feature map for layer {layer_info} has unexpected dimensions {feature_map.shape}. Skipping GAP.")
             continue

        pool_kernel_size = feature_map.size()[2:]
        if 0 in pool_kernel_size:
             warnings.warn(f"        Pooling kernel size contains 0 for layer {layer_info} ({pool_kernel_size}). Skipping GAP.")
             continue

        pooled_features = nn.functional.avg_pool3d(feature_map, kernel_size=pool_kernel_size)
        feature_vector = pooled_features.squeeze(-1).squeeze(-1).squeeze(-1) # Shape [batch_size, channels]

        if feature_vector.shape[0] == 1:
             feature_vector = feature_vector.squeeze(0) # Shape [channels]

        extracted_features[layer_info] = feature_vector.cpu().numpy() # Convert to NumPy and move to CPU for saving


    return extracted_features

# --- Main Script to Process One Volume and Save Features ---

if __name__ == "__main__":
    # --- Configuration ---
    # Directory containing your single volume and its mask files
    data_dir = './my_medical_scan/' # Example: Contains my_scan.nii.gz, my_scan_tibia.nii.gz, etc.

    # Base name of your volume file (without extension or region suffix)
    volume_base_name = 'my_scan' # Example: Your volume is named 'my_scan.nii.gz'

    # List of regions you have separate masks for (corresponding to suffixes like '_tibia')
    regions_to_process = ['tibia', 'femur', 'background']

    # Expected number of frames/slices (CNN's depth) *after* padding/cropping of regions
    # This MUST match the frame_nb used when inflating the model
    input_frame_nb_for_cnn = 64 # Example

    # Define the layers you want to extract features from
    layers_to_get = ['last', 'third_last', 'fifth_last'] # Example targets

    # Output directory to save the features
    output_dir = './extracted_features/'
    os.makedirs(output_dir, exist_ok=True) # Create output directory if it doesn't exist

    # --- Load and Inflate the Model ---
    try:
        model_2d = models.densenet121(pretrained=True)
        # Inflate with the target depth and 1 input channel for grayscale
        i3d_densenet_model = inflate_densenet121(
            model_2d,
            frame_nb=input_frame_nb_for_cnn,
            input_channels=1 # Specify 1 input channel for grayscale
        )
        print("Model loaded and inflated successfully with 1 input channel.")
        # Optional: Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        i3d_densenet_model.to(device)
        print(f"Model moved to device: {device}")

    except Exception as e:
        print(f"Error loading or inflating model: {e}")
        exit() # Exit if model setup fails

    # --- Process the single volume ---
    volume_path = os.path.join(data_dir, f'{volume_base_name}.nii.gz')
    if not os.path.exists(volume_path):
        volume_path = os.path.join(data_dir, f'{volume_base_name}.nii') # Try .nii
        if not os.path.exists(volume_path):
             print(f"\nOriginal volume file not found for base name {volume_base_name}. Exiting.")
             exit()

    print(f"\nProcessing volume: {volume_base_name}")
    all_region_features_for_volume = {} # Store features for this volume across all regions

    try:
        # Load the original volume data once
        volume_img = nib.load(volume_path)
        volume_data = volume_img.get_fdata().astype(np.float32)
        print(f"  Loaded original volume data with shape: {volume_data.shape}")
        # Optional: Move volume data to GPU before preprocessing if needed
        # volume_data_tensor = torch.from_numpy(volume_data).to(device)

        # Process each region for this volume
        for region in regions_to_process:
            mask_path = os.path.join(data_dir, f'{volume_base_name}_{region}.nii.gz')
            if not os.path.exists(mask_path):
                 mask_path = os.path.join(data_dir, f'{volume_base_name}_{region}.nii') # Try .nii
                 if not os.path.exists(mask_path):
                      print(f"    Mask file not found for region {region} at {mask_path}. Skipping region.")
                      all_region_features_for_volume[region] = {} # Store empty features for this region
                      continue

            # Load the mask data
            mask_img = nib.load(mask_path)
            mask_data = mask_img.get_fdata() # Mask data can be int or bool

            # Extract features from the masked region
            extracted_feats = extract_features_from_masked_region(
                i3d_densenet_model,
                volume_data, # Use the original NumPy data
                mask_data,
                region_name=region,
                frame_nb=input_frame_nb_for_cnn,
                target_layers=layers_to_get
            )
            all_region_features_for_volume[region] = extracted_feats

        print(f"  Finished processing regions for volume {volume_base_name}")

        # --- Save the extracted features ---
        output_filepath = os.path.join(output_dir, f'{volume_base_name}_features.pkl')
        with open(output_filepath, 'wb') as f:
            pickle.dump(all_region_features_for_volume, f)
        print(f"  Saved extracted features for volume {volume_base_name} to {output_filepath}")


    except (FileNotFoundError, ValueError, RuntimeError, TypeError) as e:
        print(f"  Error processing volume {volume_base_name}: {e}")
        # Continue wouldn't apply here as we only have one volume, but good for error handling

    print("\n--- Region Feature Extraction Complete ---")

    # You can now load the features later using pickle:
    # with open('./extracted_features/my_scan_features.pkl', 'rb') as f:
    #     loaded_features = pickle.load(f)
    #
    # # Access features, e.g., for Tibia, last Conv3d:
    # tibia_feats = loaded_features['tibia']
    # for key, feat in tibia_feats.items():
    #     if key[0] == 'last':
    #          print("\nLoaded Tibia features (last Conv3d):", feat)
    #          print("Shape:", feat.shape)

Model loaded and inflated successfully.

Processing region: Tibia
  Processing volume: tibia.nii.gz


/var/folders/kb/f1k4_th17h992yvkrm2v50d40000gn/T/ipykernel_3536/216718156.py:115: UserWarning: NIfTI volume depth (512) does not match frame_nb (64) used during model inflation. This might cause issues or requires padding/cropping.
  warnings.warn(f"NIfTI volume depth ({current_volume_depth}) does not match frame_nb ({frame_nb}) "
/var/folders/kb/f1k4_th17h992yvkrm2v50d40000gn/T/ipykernel_3536/216718156.py:153: UserWarning: Invalid target layer format: third_last. Use 'last', 'second_last', etc.
  warnings.warn(f"Invalid target layer format: {target}. Use 'last', 'second_last', etc.")
/var/folders/kb/f1k4_th17h992yvkrm2v50d40000gn/T/ipykernel_3536/216718156.py:153: UserWarning: Invalid target layer format: fifth_last. Use 'last', 'second_last', etc.
  warnings.warn(f"Invalid target layer format: {target}. Use 'last', 'second_last', etc.")


Loaded volume shape: (512, 512, 216), Prepared tensor shape: torch.Size([1, 1, 512, 512, 216])


Model loaded and inflated successfully.
  Loaded original volume data with shape: (512, 512, 216)
      Processing region: tibia
        Preprocessed tensor shape: torch.Size([1, 1, 16, 101, 216])
  Error processing volume:       Forward pass failed for region tibia: Given groups=1, weight of size [64, 3, 7, 7, 7], expected input[1, 1, 16, 101, 216] to have 3 channels, but got 1 channels instead

--- Region Feature Extraction Complete ---


/var/folders/kb/f1k4_th17h992yvkrm2v50d40000gn/T/ipykernel_3536/4205872307.py:96: UserWarning:         Cropped region depth (116) does not match target frame_nb (16). Applying padding/cropping.
  warnings.warn(f"        Cropped region depth ({current_region_depth}) does not match target frame_nb ({target_depth}). Applying padding/cropping.")
/var/folders/kb/f1k4_th17h992yvkrm2v50d40000gn/T/ipykernel_3536/4205872307.py:159: UserWarning: Invalid target layer format: third_last. Use 'last', 'second_last', etc.
  warnings.warn(f"Invalid target layer format: {target}. Use 'last', 'second_last', etc.")
/var/folders/kb/f1k4_th17h992yvkrm2v50d40000gn/T/ipykernel_3536/4205872307.py:159: UserWarning: Invalid target layer format: fifth_last. Use 'last', 'second_last', etc.
  warnings.warn(f"Invalid target layer format: {target}. Use 'last', 'second_last', etc.")


In [27]:
region_features

[('Background_sample_volume.nii.gz',
  {('conv2',
    'Conv3d'): tensor([ 0.0051,  0.0122, -0.0055, -0.0065,  0.0012, -0.0178, -0.0320, -0.0056,
           -0.0361, -0.0274, -0.0244, -0.0153, -0.0003, -0.0026, -0.0136,  0.0120,
            0.0023,  0.0165, -0.0097, -0.0062, -0.0051,  0.0130, -0.0266,  0.0109,
           -0.0180, -0.0190, -0.0017,  0.0228, -0.0170,  0.0546,  0.0062,  0.0015])})]